In [1]:
# Homework 6 Template
# Adrien Masini

In [2]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass



In [ ]:
def OrbitCOM(galaxy, start, end, n=5) :
    """function that loops over all the desired snapshots to compute the COM pos and vel as a function of time.
    inputs: galaxy the name of the galaxy, e.g. “MW”, start the number of the first snapshot to be read in., end the number of the last snapshot to be read in., n an integer indicating the intervals over which you will return the COM.
          
    returns: 
    """
    
    # compose the filename for output
    
    fileout = "Orbit_"+galaxy+".txt"
    
    #  set tolerance and VolDec for calculating COM_P in CenterOfMass
    # for M33 that is stripped more, use different values for VolDec
    
    if (galaxy == 'M33'):
        delta = 0.1
        VolDec=4
    else :
        delta = 0.1
        VolDec = 2
        
    
    # generate the snapshot id sequence 
    # it is always a good idea to also check if the input is eligible (not required)
    
    snap_ids = np.arange(start, end+1, step=n)
    
    # initialize the array for orbital info: t, x, y, z, vx, vy, vz of COM
    
    orbit = np.zeros([snap_ids.size, 7])
    
    for i, snap_id in enumerate(snap_ids):
        
        # compose the data filename (be careful about the folder)
        
        # Determine Filename
        # add a string of the filenumber to the value "000"
        ilbl = '000' + str(snap_id)
        # remove all but the last 3 digits
        ilbl = ilbl[-3:]
        # create filenames
        filename='%s_'%(galaxy) + ilbl + '.txt'
        
        # Initialize an instance of CenterOfMass class, using disk particles
        
        COM = CenterOfMass(filename, 2)
        
        # Store the COM pos and vel. Remember that now COM_P required VolDec
       
        COM_P = COM.COM_P(delta, VolDec)
        COM_V = COM.COM_V(COM_P[0],COM_P[1],COM_P[2])
        
        # store the time, pos, vel in ith element of the orbit array,  without units (.value) 
        # note that you can store 
        # a[i] = var1, *tuple(array1)
        
        orbit[i] = COM.time.value/1000, *tuple(COM_P.value), *tuple(COM_V.value)
        
        
        # print snap_id to see the progress
        print(snap_id)
   
        
    # write the data to a file
    # we do this because we don't want to have to repeat this process 
    # this code should only have to be called once per galaxy.
    np.savetxt(fileout, orbit, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

In [ ]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first! 

OrbitCOM('MW', 0, 800, n=5)
OrbitCOM('M31', 0, 800, n=5)
OrbitCOM('M33', 0, 800, n=5)


0
5
10


In [ ]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt

Orbit_MW_data = np.genfromtxt("Orbit_MW.txt")
Orbit_M31_data = np.genfromtxt("Orbit_M31.txt")
Orbit_M33_data = np.genfromtxt("Orbit_M33.txt")

In [ ]:
# function to compute the magnitude of the difference between two vectors 
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit  


def V_C(vc1, vc2):
    
    Sub = vc1-vc2
    
    return np.sqrt(Sub[:,0]**2+Sub[:,1]**2+Sub[:,2]**2)

In [ ]:
# Determine the magnitude of the relative position and velocities 

# For MW and M31
MW_M31 = V_C(OrbMW_data[:, 1:4], OrbM31_data[:, 1:4])
VMW_M31 = V_C(OrbMW_data[:, 4:8], OrbM31_data[:, 4:8])

# For M33 and M31
M33_M31 = V_C(OrbM33_data[:, 1:4], OrbM31_data[:, 1:4])
VM33_M31 = V_C(OrbM33_data[:, 4:8], OrbM31_data[:, 4:8])

In [ ]:
# Plot the Orbit of the galaxies 

# MW_M31 SEPARATION

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
plt.plot(OrbitMW_data[:,0], MWM31, color='blue', linewidth=4, label='MW_M31 Separation')

# Add axis labels
plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Separation (kpc)', fontsize=22)

label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# Legend
legend = ax.legend(loc='upper right',fontsize='x-large')

## M33_M31 SEPARATION
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
plt.plot(OrbitM31_data[:,0], M33M31, color='white', linewidth=4, label='M33_M31 Separation')

# Axis labels
plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Separation (kpc)', fontsize=22)

label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# add a legend
legend = ax.legend(loc='upper right',fontsize='x-large')

In [ ]:
# Plot the orbital velocities of the galaxies 

# MW - M31 ORBITAL VEL

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
plt.plot(OrbitMW_data[:,0], VMWM31, color='salmon', linewidth=4, label='MW-M31 Orbital Vel')

# Add axis labels
plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Relative Velocity (km/s)', fontsize=22)


#adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# add a legend with some customizations.
legend = ax.legend(loc='upper right',fontsize='x-large')

# M33_M31 ORBITAL VELOCITY

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
plt.plot(OrbitM31_data[:,0], VM33M31, color='plum', linewidth=4, label='M33-M31 Orbital Vel')

# Add axis labels
plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Relative Velocity (km/s)', fontsize=22)
#adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# add a legend with some customizations.
legend = ax.legend(loc='upper right',fontsize='x-large')